<a href="https://colab.research.google.com/github/PatrickLewandowski/SoundEventDetection/blob/master/manuel/Deliveries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-bigquery-datatransfer
!pip install --upgrade google-cloud-storage

In [ ]:
#@title Click run to autentication and setup
# Load libraries 
from google.colab import auth
from google.cloud.bigquery import SchemaField
from google.cloud import bigquery
import time
import pandas_gbq
from datetime import timedelta, date
from google.cloud.exceptions import NotFound
import pandas as pd
import ipywidgets as widgets
import yaml
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


# **Notebook**
This notebook has been creaed with the purpose of enabling a smoother creation of resources on the delivery project(telia-ddi-delivery).

Considerations:

1. The name of the dataset must be written in lower case and include a preffix with the country which is meant to be used. I.E: no_X,fin_x,swe_X,...

2. The Dataset_name variable will be used as the ID for the schedule query and the bucket.  

3. All the parameters need to be filled-up before running any other cell.  

4. The name of the dataset/bucket/schedule query* will be created based on the following pattern:{country}_{ticket_number}_{customer}.

5. If the delivery uses schedule query and runs on weekly basis the day of week that runs which be taken from the start_date. I.E Setting up a weekly delivery on the 2020-02-08 means that it would run every Monday.

** The schedule query will have extra suffix with the table name at the end {country}_{ticket_number}_{customer}_{table_name}


In [ ]:
#@title Parameters for creating bucket/dataset/schedule query
#Dataset names cannot contain spaces or special characters such as -, &, @, or %.
ticket_number = '123421' #@param {type:"string"}
customer= 'trakkif' #@param {type:"string"}
country = 'no' #@param ["swe", "no",'dk','fin','est']
Frequency = 'weekly' #@param ["weekly", "daily","monthly"]
Recurrent= 'yes' #@param ["yes", "no"]
tableau= 'yes' #@param ["yes", "no"]
csv= 'yes' #@param ["yes", "no"]
start_date = '2021-03-12' #@param {type:"date"}
end_date = '2021-03-26' #@param {type:"date"}
hour = 14 #@param {type:"slider", min:0, max:24, step:1}
customer=(customer.lower())
Dataset_name=f"{country}_{ticket_number}_{customer}"





In [ ]:
#@title Create dataset
from google.cloud import bigquery


Dataset_name=str(Dataset_name).lower()
project="telia-ddi-delivery"
# Construct a BigQuery client object.
client = bigquery.Client(project)

# TODO(developer): Set dataset_id to the ID of the dataset to create.
# dataset_id = "{}.your_dataset".format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(f"{project}.{Dataset_name}")

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "EU"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

# Construct a BigQuery client object.
client = bigquery.Client(project)

# TODO(developer): Set dataset_id to the ID of the dataset to fetch.
# dataset_id = "your-project.your_dataset"

dataset = client.get_dataset(Dataset_name)  # Make an API request.
dataset.labels = {"frequency": Frequency, "recurrent":Recurrent,"tableau":tableau,"csv":csv,"customer":customer}
dataset = client.update_dataset(dataset, ["labels"])  # Make an API request.

print("Labels added to {}".format(Dataset_name))
print(dataset.labels)

Created dataset telia-ddi-delivery.no_123421_trakkif
Labels added to no_123421_trakkif
{'frequency': 'weekly', 'recurrent': 'yes', 'tableau': 'yes', 'csv': 'yes', 'customer': 'trakkif'}


In [ ]:
#@title Modify labels of the dataset
from google.cloud import bigquery


Dataset_name=str(Dataset_name).lower()
project="telia-ddi-delivery"

client = bigquery.Client(project)

# TODO(developer): Set dataset_id to the ID of the dataset to fetch.
# dataset_id = "your-project.your_dataset"

dataset = client.get_dataset(Dataset_name)  # Make an API request.
dataset.labels = {"frequency": Frequency, "recurrent":Recurrent,"tableau":tableau,"csv":csv,"customer":customer}
dataset = client.update_dataset(dataset, ["labels"])  # Make an API request.

print("Labels added to {}".format(Dataset_name))
print(dataset.labels)

Labels added to test_323232
{'frequency': 'weekly', 'recurrent': 'yes', 'tableau': 'yes', 'one_off': 'yes', 'csv': 'yes', 'customer': 'trakkif'}


In [ ]:
#@title Create bucket
from google.cloud import storage
import pprint

Dataset_name=str(Dataset_name).lower()
project="telia-ddi-delivery"
# Instantiates a client
storageClient = storage.Client(project)
bucket = storageClient.bucket(Dataset_name)
#bucket.storage_class = "COLDLINE"
bucket.create(location="EU",)
print("Bucket {} created.".format(Dataset_name))

storage_client = storage.Client(project)

bucket = storage_client.get_bucket(Dataset_name)
bucket.labels = {"frequency": Frequency, "recurrent":Recurrent,"tableau":tableau,"csv":csv,"customer":customer}
bucket.iam_configuration.uniform_bucket_level_access_enabled = True
bucket.patch()

print("Updated labels on {}.".format(Dataset_name))
pprint.pprint(bucket.labels)

storage_client = storage.Client(project)
bucket = storage_client.get_bucket(Dataset_name)
bucket.iam_configuration.uniform_bucket_level_access_enabled = True
bucket.patch()

#print(
 #   "Uniform bucket-level access was enabled for {}.".format(bucket.name)
#)

Bucket no_123421_trakkif created.
Updated labels on no_123421_trakkif.
{'csv': 'yes',
 'customer': 'trakkif',
 'frequency': 'weekly',
 'recurrent': 'yes',
 'tableau': 'yes'}


In [ ]:
#@title Modify labels of the bucket
from google.cloud import storage
import pprint


Dataset_name=str(Dataset_name).lower()
project="telia-ddi-delivery"
# Instantiates a client
storage_client = storage.Client(project)

bucket = storage_client.get_bucket(Dataset_name)
bucket.labels = {"frequency": Frequency, "recurrent":Recurrent,"tableau":tableau,"csv":csv,"customer":customer}
bucket.patch()
print("Updated labels on {}.".format(bucket.name))
pprint.pprint(bucket.labels)


storage_client = storage.Client(project)
bucket = storage_client.get_bucket(Dataset_name)
bucket.iam_configuration.uniform_bucket_level_access_enabled = True
bucket.patch()

#print(
 #   "Uniform bucket-level access was enabled for {}.".format(bucket.name)
#)





Updated labels on test_1234412211231.
{'csv': 'yes',
 'customer': 'trakkif',
 'frequency': 'weekly',
 'one_off': 'yes',
 'recurrent': 'yes',
 'tableau': 'yes'}
Uniform bucket-level access was enabled for test_1234412211231.


In [ ]:
#@title Add users to the bucket
#Dataset names cannot contain spaces or special characters such as -, &, @, or %.
email_list = ["maria.isaksson@reitanconvenience.se" ]#@param {type:"raw"}







In [ ]:
#@title Click run to print a yaml file and user template for the bucket

#empty variable for email list iteration
email = ""
index = 0

#loop through email list specified, use inserted parameters
for email in email_list:
    file_name = email.split('@')[0].lower() + '.yml'
    index += 1
    output = f"""meta: 'Access bucket'
gcs:
  - bucket: "{Dataset_name}"
    expires: "{end_date}"
    editor: true
email: "{email}"
"""
    print(output)

 


meta: 'Access bucket'
gcs:
  - bucket: "no_123421_trakkif"
    expires: "2021-03-03"
    editor: true
email: "maria.isaksson@reitanconvenience.se"



In [ ]:
#@title Click run to create a yaml file and user template for the bucket
from google.colab import drive
drive.mount('/content/gdrive')
#empty variable for email list iteration
email = ""
index = 0

#loop through email list specified, use inserted parameters
for email in email_list:
    file_name = email.split('@')[0].lower() + '.yml'
    index += 1
    output = f"""meta: 'Access bucket'
gcs:
  - bucket: "{Dataset_name}"
    expires: "{end_date}"
    editor: true
email: "{email}"
"""
    #print(output)
    #make the yml file here using file_name and output
    with open(f'/content/gdrive/My Drive/{file_name}', 'w') as outfile:
      print(outfile)
      #print(yaml.safe_load(yaml.dump(output,default_flow_style=False)))
      write = yaml.safe_load(yaml.dump(output,default_flow_style=False))
      outfile.write(write)
      print(f"Wrote file {file_name} to Drive")
      time.sleep(2)
      print(f"File {index} done: {file_name}")
      print()
 


# **Schedule queries**
The main purpose of the schedule queries is to grant you the possiblity of creating recurrent deliveries either way on Tableau or CSV.

Considerations:


1.   The schedules query must have on top of the query a create or replace table statment to recreate the table.

2.   If the frequency is daily the query shouldn't run more than 500GB.

3.   If the frequency is weekly the query shouldn't run more than 2TB.

4.   The schedule queries can generate CSV files in bucket by including the following chunk of code at the top of the query:

5. Please set-up the table name and file name parameter in case that the delivery requires CSV

In [ ]:
#@title Parameters for the schedule query
#Dataset names cannot contain spaces or special characters such as -, &, @, or %.
table_name = 'INPUT_TABLE_NAME' #@param {type:"string"}
file_name = 'INPUT_FILE_NAME' #@param {type:"string"}

SyntaxError: ignored

In [ ]:
##########Example SQ with CSV############
###Write file_name & table_name
###DON'T CHANGE THE dataset_name variable
### Fill-up table_name parameters & file_name###
Dataset_name=str(Dataset_name).lower()
query_string = f"""
/* ddi-meta
project_id: telia-ddi-delivery
source_dataset: {Dataset_name}
source_table: {table_name}
output_destination_uri: gs://{Dataset_name}/{file_name}_*.csv
*/
create or replace table `telia-ddi-delivery.{Dataset_name}.{table_name}`
as
#Example query, pls replace this part of the code with your actual query#
SELECT *
FROM `telia-ddi-est-dev.Estonia_COVID.covidhomezone`;

"""

The table name and the file name need to be modified accordingly to the deliver.

In [ ]:
##########Example SQ without CSV############
###Write file_name & table_name
###DON'T CHANGE THE dataset_name variable
Dataset_name=str(Dataset_name).lower()
table_name="INPUT_TABLE_NAME"# To be changed
query_string = f"""

create or replace table `telia-ddi-delivery.{Dataset_name}.{table_name}`
as
#Example query, pls replace this part of the code with your actual query#
SELECT
  CURRENT_TIMESTAMP() as current_time,
  @run_time as intended_run_time,
  @run_date as intended_run_date,
  17 as some_integer;
"""

In [ ]:
#@title Create schedule query
from google.cloud import bigquery_datatransfer
import time
transfer_client = bigquery_datatransfer.DataTransferServiceClient()
from google.protobuf.timestamp_pb2 import Timestamp

import datetime
date_time_str = f'{start_date} {hour}:00:00.0'
print(f"Starting date: {date_time_str} ")
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
start_timestamp=datetime.datetime.timestamp(date_time_obj)
start_timestamp = Timestamp(                    seconds=int(start_timestamp))


day_of_week=date_time_obj.strftime('%A')

date_time_str = f'{end_date} {hour}:00:00.0'
print(f"end date: {date_time_str} ")
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
end_timestamp=datetime.datetime.timestamp(date_time_obj)
end_timestamp = Timestamp(                    seconds=int(end_timestamp))

# The project where the query job runs is the same as the project
# containing the destination dataset.
project_id = "telia-ddi-delivery"
Dataset_name=str(Dataset_name).lower()

# Use standard SQL syntax for the query.

if "daily" in Frequency:
  schedule=f"every 24 hours"
elif "weekly":
  schedule=f"every {day_of_week} {hour}:00"
else:
  schedule=f"1 of month 00:00"  
notification_pubsub_topic=""
if "ddi-meta" in query_string:
  notification_pubsub_topic="projects/telia-ddi-delivery/topics/bq-export-table-topic"



transfer_client = bigquery_datatransfer.DataTransferServiceClient()
project_id = "telia-ddi-delivery"
destination_dataset_id=""
parent = transfer_client.common_project_path(project_id)
transfer_config = bigquery_datatransfer.TransferConfig(
  destination_dataset_id="",
  display_name=f"{Dataset_name}_{table_name}",
  data_source_id="scheduled_query",
  params={
      "query": query_string,
  },
  schedule=schedule,
  notification_pubsub_topic=notification_pubsub_topic,
  dataset_region="EU",
  email_preferences={"enable_failure_email":True},
  schedule_options= {"start_time":start_timestamp,"end_time":end_timestamp} 
)


parent = "projects/telia-ddi-delivery/locations/EU"

transfer_config = transfer_client.create_transfer_config(
    bigquery_datatransfer.CreateTransferConfigRequest(
        parent=parent, transfer_config=transfer_config,
    )
)
print("Created scheduled query '{}'".format(transfer_config.name))



In [ ]:
#@title If the above does not work run this and follow the instructions

!gcloud auth application-default login  --no-launch-browser
